# 9.6 Lab: Support Vector Machines

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score,confusion_matrix,roc_curve, auc, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import json
%matplotlib inline

In [ ]:
# support function to plot the decision boundary of svc and highlight the support vectors
def plot_decision_boundary(svc, X, y, h=0.021, pad=0.21):
    x_min, x_max = X[:, 0].min() - pad, X[:, 0].max() + pad
    y_min, y_max = X[:, 1].min() - pad, X[:, 1].max() + pad
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.2)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired)

    # highlight the support vectors
    sv = svc.support_vectors_
    plt.scatter(sv[:,0], sv[:,1], c='k', marker='*', s=21, linewidths=1)
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.show()
    print('Number of support vectors: ', svc.support_.size)

## 9.6.1 Support Vector Classifier

In [ ]:
# we start from generating random dataset: following the bookm we generate a dataset with 20 observations,
# 2 features. And we divide these into two classes.
# set seed 
np.random.seed(21)
X = np.random.randn(20, 2)
y = np.repeat([-1,1], 10)
X[y==1] = X[y==1] + 1

plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=plt.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

In [ ]:
# Support Vector Classifier (i.e. support vector machine with linear kernel)
svc1 = SVC(C= 10, kernel='linear')
svc1.fit(X, y)

plot_decision_boundary(svc1, X, y)

In [ ]:
# as mentioned before, we could use dir() to see the methods of the class
# I did not find a good way to print out the summary of the SVC model.
dir(svc1)

In [ ]:
# we could take a look at the defaul parameters of the SVC model
svc1.get_params()

In [ ]:
# we use a small cost (c = 0.1). A smaller value of the cost parameter is being used, 
# we obtain a larger number of support vectors, because the margin is now wider. 
svc2 = SVC(C=0.1, kernel='linear')
svc2.fit(X, y)

plot_decision_boundary(svc2, X, y)


In [ ]:
# we could also try to tune the cost parameter (C) of the SVC model using GridSearchCV
# in this function, we need to specify cross validation folds and the metric to use for evaluation
tuned_parameters = [{'C': [0.001, 0.01, 0.1, 1, 5, 10, 100]}]
clf = GridSearchCV(SVC(kernel='linear'), tuned_parameters, cv=10, scoring='accuracy', return_train_score=True)
clf.fit(X, y)
clf.cv_results_

In [ ]:
# let us see the best parameters. 
# This is different from the results in the book, it is very likely due to the random generation of the datasetof the data
clf.best_params_

In [ ]:
# we use the same generation process to generate test data
X_test = np.random.randn(20, 2)
y_test = np.repeat([-1,1], 10)
X_test[y_test==1] = X_test[y_test==1] + 1

plt.scatter(X_test[:,0], X_test[:,1], s=70, c=y_test, cmap=plt.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

In [ ]:
# train a model with the optimal parameters
svc3 = SVC(C=1, kernel='linear')
svc3.fit(X, y)

y_pred = svc3.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred),index=svc3.classes_, columns=svc3.classes_)

In [ ]:
# now we make our data linear separable. In the book, they add another 0.5 to seperate the data. 
# here we start from the data generation process to aviod confusion.
np.random.seed(21)
X = np.random.randn(20, 2)
y = np.repeat([-1,1], 10)
X[y==1] = X[y==1] + 2.5

plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=plt.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

In [ ]:
X_test = np.random.randn(20, 2)
y_test = np.repeat([-1,1], 10)
X_test[y_test==1] = X_test[y_test==1] + 2.5

plt.scatter(X_test[:,0], X_test[:,1], s=70, c=y_test, cmap=plt.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

In [ ]:
# here seems the data is linear separable. We could use a bigger cost parameter (C = 100) to train the model.
svc4 = SVC(C=100, kernel='linear')
svc4.fit(X, y)

plot_decision_boundary(svc4, X, y)

In [ ]:
y_pred = svc4.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred),index=svc4.classes_, columns=svc4.classes_)

## 9.6.2 Support Vector Machine

In [ ]:
# generating random dataset
np.random.seed(21)
X = np.random.randn(200,2)
X[:100] = X[:100] + 2
X[101:150] = X[101:150] - 2
y = np.concatenate([np.repeat(-1, 150), np.repeat(1,50)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=2)

plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=plt.cm.Paired)
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()

In [ ]:
# in python, we can use the same svc model abd kernel to specific the kernel
# for rbf kernel, we need to specify the gamma parameter
svm = SVC(C=1.0, kernel='rbf', gamma=1)
svm.fit(X_train, y_train)

In [ ]:
plot_decision_boundary(svm, X_train, y_train)

In [ ]:
y_pred = svm.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred),index=svm.classes_, columns=svm.classes_)

In [ ]:
# increasing C parameter which increases more flexibility
svm2 = SVC(C=100, kernel='rbf', gamma=1.0)
svm2.fit(X_train, y_train)
plot_decision_boundary(svm2, X_train, y_train)

In [ ]:
"""
The above decision boundary seems overfitting. We can compute the test accuracy of the model to
see whether that is the case. 

The model (c = 1) yields a test accuracy of 0.85; the model(c = 100) yields a test accuracy of 0.77.
"""
y_pred = svm2.predict(X_test)
pd.DataFrame(confusion_matrix(y_test, y_pred),index=svm2.classes_, columns=svm2.classes_)

In [ ]:
# set the parameters by cross-validation
tuned_parameters = [{'C': [0.01, 0.1, 1, 10, 100],
                     'gamma': [0.5, 1,2,3,4]}]
clf = GridSearchCV(SVC(kernel='rbf'), tuned_parameters, cv=10, scoring='accuracy', return_train_score=True)
clf.fit(X_train, y_train)
clf.cv_results_

In [ ]:
# let us see the best parameters.
clf.best_params_

In [ ]:
# confusion matrix for the best model
confusion_matrix(y_test, clf.best_estimator_.predict(X_test))

In [ ]:
# calculate the test accuracy
clf.best_estimator_.score(X_test, y_test)

## 9.6.3 ROC Curves

In [ ]:
svm3 = SVC(C=1, kernel='rbf', gamma=2)
svm3.fit(X_train, y_train)

# we train another model flexible model
svm4 = SVC(C=1, kernel='rbf', gamma=50)
svm4.fit(X_train, y_train)

y_train_score3 = svm3.decision_function(X_train)
y_train_score4 = svm4.decision_function(X_train)

false_pos_rate3, true_pos_rate3, _ = roc_curve(y_train, y_train_score3)
roc_auc3 = auc(false_pos_rate3, true_pos_rate3)

false_pos_rate4, true_pos_rate4, _ = roc_curve(y_train, y_train_score4)
roc_auc4 = auc(false_pos_rate4, true_pos_rate4)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(14,6))
ax1.plot(false_pos_rate3, true_pos_rate3, label='SVM $\gamma = 1$ ROC curve (area = %0.2f)' % roc_auc3, color='b')
ax1.plot(false_pos_rate4, true_pos_rate4, label='SVM $\gamma = 50$ ROC curve (area = %0.2f)' % roc_auc4, color='r')
ax1.set_title('Training Data')

y_test_score3 = svm3.decision_function(X_test)
y_test_score4 = svm4.decision_function(X_test)

false_pos_rate3, true_pos_rate3, _ = roc_curve(y_test, y_test_score3)
roc_auc3 = auc(false_pos_rate3, true_pos_rate3)

false_pos_rate4, true_pos_rate4, _ = roc_curve(y_test, y_test_score4)
roc_auc4 = auc(false_pos_rate4, true_pos_rate4)

ax2.plot(false_pos_rate3, true_pos_rate3, label='SVM $\gamma = 1$ ROC curve (area = %0.2f)' % roc_auc3, color='b')
ax2.plot(false_pos_rate4, true_pos_rate4, label='SVM $\gamma = 50$ ROC curve (area = %0.2f)' % roc_auc4, color='r')
ax2.set_title('Test Data')

for ax in fig.axes:
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([-0.05, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.legend(loc="lower right")

""" 
From the plots below, we can see that the model with gamma = 50 is overfitting the training data 
(i.e. the training metric is much better than the test metric).
"""

## 9.6.4 SVM with Multiple Classes

In [ ]:
# generate the previously used random dataset
np.random.seed(21)
X = np.random.randn(200,2)
X[:100] = X[:100] + 2
X[101:150] = X[101:150] - 2
y = np.concatenate([np.repeat(-1, 150), np.repeat(1,50)])

# adding another class to the dataset, I used a different offset to separate the classes better
XX = np.vstack([X, np.random.randn(50,2)])
yy = np.hstack([y, np.repeat(0,50)])
XX[yy==0, 1] = XX[yy==0, 1] + 6

plt.scatter(XX[:,0], XX[:,1], s=70, c=yy, cmap=plt.cm.prism)
plt.xlabel('XX1')
plt.ylabel('XX2')

In [ ]:
# fit the svm model 
svm5 = SVC(C=10, kernel='rbf', gamma=1)
svm5.fit(XX, yy)
plot_decision_boundary(svm5, XX, yy)

## 9.6.5 Application to Gene Expression Data

In [ ]:
# I saved the gene expression data as a json file, in python we could load the json file using the json library
# after reading in the data, we can use the data is same as a dictionary, we can use the keys to access the data
# import json
f = open('./data/Khan.json',)
Khan = json.load(f)
print(Khan.keys())

In [ ]:
X_train = np.array(Khan['xtrain'])
y_train = np.array(Khan['ytrain'])
X_test = np.array(Khan['xtest'])
y_test = np.array(Khan['ytest'])

In [ ]:
# take a look at the data, we will notice there are 4 classes
np.unique(y_train)

In [ ]:
svm6 = SVC(C = 10, kernel='linear')
svm6.fit(X_train, y_train)

In [ ]:
""" 
We see below that the model is perfect on training data. In fact, this is not surprising, 
because the large number of variables relative to the number of observations implies that 
it is easy to find hyperplanes that fully separate the classes. We are most interested not 
in the support vector classifier’s performance on the training observations, but rather its 
performance on the test observations.
"""
print('train accuracy', svm6.score(X_train, y_train))
y_pred = svm6.predict(X_test)
print('test accuracy', svm6.score(X_test, y_test))

In [ ]:
# End of Chapter 9